# Prep filtered scaffold sets for distributed design

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/projects/crispy_shifty/notebooks
running on node: dig136


### Set working directory to the root of the crispy_shifty repo

In [2]:
os.chdir("/projects/crispy_shifty")

### Load a dataframe of filtered scaffolds and associated metadata
These scaffolds had AF2 run on them; for their best quality prediction out of the 5 AF2 ptm models they have > 92 plddt and < 1.5 RMSD to design.  
We will also make the task generator here

In [3]:
def create_tasks(scaffolds, options):
    metadata_to_keep = [
        "pdb",
        "topo",
        "best_model",
        "best_average_plddts",
        "best_ptm",
        "best_rmsd_to_input",
        "best_average_DAN_plddts",
        "scaffold_type",
    ]
    for _, row in scaffolds.iterrows():
        metadata = dict(row)
        metadata = {k: v for k, v in metadata.items() if k in metadata_to_keep}
        pdb_path = metadata["pdb"]
        tasks = {}
        tasks["extra_options"] = options
        tasks["metadata"] = metadata
        tasks["pdb_path"] = pdb_path
        yield tasks


scaffolds = pd.read_csv(
    os.path.join(os.getcwd(), "scaffolds/00_filter_scaffold_sets/all_filtered.csv")
)

### Domesticate the scaffolds by trimming off leading and trailing loops, designing away disulfides and adding metadata to the output pdb.bz2s. 

In [4]:
# Python standard library                                                       
import os
import pwd
import socket
import sys
# 3rd party library imports                                                     
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
# Rosetta library imports                                                       
from pyrosetta.distributed.cluster.core import PyRosettaCluster
# Custom library imports
sys.path.insert(0, os.getcwd())
from crispy_shifty.protocols.cleaning import remove_terminal_loops, redesign_disulfides # the functions we will distribute


print("run the following from your local terminal to port forward the dashboard to localhost")
print(
    f"ssh -L 8000:localhost:8787 {pwd.getpwuid(os.getuid()).pw_name}@{socket.gethostname()}"
)
print("dashboard is now visible at localhost:8000")
print(f"can also view dashboard at {socket.gethostname()}:8787 without port forwarding")
options = {
    "-out:level": "200", # warning outputs only
    "-corrections::beta_nov16": "true",
    "-detect_disulf": "false",
    "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
    "-indexed_structure_store:fragment_store": "/home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5",
}
output_path = os.path.join(os.getcwd(), "scaffolds/01_prep_inputs")
os.makedirs(output_path, exist_ok=True)

if __name__ == "__main__":
    # configure SLURM cluster as a context manager
    with SLURMCluster(
        cores=1,
        processes=1,
        job_cpu=1,
        memory="12GB",
        queue="medium",
        walltime="23:30:00",
        death_timeout=120,
        local_directory="$TMPDIR/dask",
        log_directory="/mnt/home/pleung/logs/slurm_logs",
        extra=["--lifetime", "23h", "--lifetime-stagger", "5m"],
        env_extra=[
            "export PYTHONPATH='$PATH:{0}'".format(
                os.path.dirname("/software/conda/condabin")
            )
        ],
    ) as cluster:
        print(cluster.job_script())
        # scale between 1-300 workers,
        cluster.adapt(
            minimum=1,
            maximum=300,
            wait_count=999,  # Number of consecutive times that a worker should be suggested for removal it is removed
            interval="5s",  # Time between checks
        )
        # setup a client to interact with the cluster as a context manager
        with Client(cluster) as client:
            print(client)
            client.upload_file(os.path.join(os.getcwd(), "crispy_shifty/protocols/cleaning.py")) # upload the script that contains the functions to distribute
            PyRosettaCluster(
                client=client,
                logging_level="WARNING",
                output_path=output_path,
                project_name="crispy_shifty",
                scratch_dir=output_path,
                simulation_name="notebooks_01_prep_inputs",
                tasks=create_tasks(scaffolds, options),
            ).distribute(protocols=[remove_terminal_loops, redesign_disulfides])
            client.close()
        cluster.scale(0)
        cluster.close()
    print("distributed run complete")

NameError: name 'Required' is not defined

In [6]:
os.path.join(os.getcwd(), "crispy_shifty/protocols/cleaning.py")

'/mnt/projects/crispy_shifty/crispy_shifty/protocols/cleaning.py'

### Unused blocks

In [3]:
sys.path.insert(
    0, "/home/pleung/projects/crispy_shifty/protocols"
)  # TODO change to projects dir for production
from cleaning import remove_terminal_loops, redesign_disulfides

remove_terminal_loops(
    None,
    pdb_path="/net/shared/scaffolds/pre_scaffold_DB/bcov_JHR/JHR_bd1_555.pdb",
    metadata={
        "pdb": "/net/shared/scaffolds/pre_scaffold_DB/bcov_JHR/JHR_bd1_555.pdb",
        "topo": "HHHH",
        "best_model": 2,
        "best_average_plddts": 96.0650150399,
        "best_ptm": 0.8458813818,
        "best_rmsd_to_input": 1.2088154485,
        "best_average_DAN_plddts": 0.947265625,
        "scaffold_type": "bcov_JHR",
    },
)

https://docs.anaconda.com/anaconda/install

core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue ARG:CtermProteinFull 107
LHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHL
